In [ ]:
from truedata_ws.websocket.TD import TD
import pandas as pd
import pytz
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout,Activation
import tensorflow as tf

In [ ]:
from datetime import time as dt
from datetime import datetime,timedelta
from time import time, sleep
from statistics import mean
import mysql.connector

In [3]:
df_1 = pd.read_csv('data_1.csv')
df_1 = pd.DataFrame(df_1)
df_1.tail()

,Date,Time,close,volume
937734,20220210,15:26:00,39014.00,20875
937735,20220210,15:27:00,39009.30,19925
937736,20220210,15:28:00,39011.25,32075
937737,20220210,15:29:00,39020.00,21200
937738,20220210,15:30:00,39035.00,425


In [4]:
training_size= int(0.85*len(df_1))
test_size=len(df_1)-training_size
train_data,test_data=df_1[0:training_size],df_1[training_size:len(df_1)]
print(train_data.shape)
print(test_data.shape)

(797078, 4)
(140661, 4)


In [5]:
df11 = train_data['close']
df22 = train_data['volume']

In [6]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle

df11 = df11.values
df22 = df22.values

df11 = df11.reshape((len(df11), 1))
df22 = df22.reshape((len(df22), 1))

scaler = MinMaxScaler(feature_range=(0, 1))
scaler1 = MinMaxScaler(feature_range=(0, 1))
df11_scaled = scaler.fit_transform(df11)
df22_scaled = scaler1.fit_transform(df22)

dataset_stacked = np.hstack((df11_scaled, df22_scaled))

In [7]:
username = 't****2'
password = 'd*****2'
td_app = TD(username, password, live_port=None)
symbol = 'BANKNIFTY-I'
barsize = '1min'
hist_data = td_app.get_historic_data(symbol, duration='2 D', bar_size=barsize)
hist_data = pd.DataFrame(hist_data)
print(hist_data)
td_app.disconnect()


(2022-03-22 06:41:03,707) WARNING :: Connected successfully to TrueData Historical Data Service...  (PID:4836 Thread:2832)


                   time         o         h         l         c      v  \
0   2022-03-21 09:15:00  36510.00  36599.95  36432.40  36478.70  76050   
1   2022-03-21 09:16:00  36483.60  36485.05  36357.95  36372.90  49750   
2   2022-03-21 09:17:00  36358.00  36364.60  36282.45  36282.45  51075   
3   2022-03-21 09:18:00  36293.45  36324.75  36290.00  36307.05  36600   
4   2022-03-21 09:19:00  36300.05  36324.00  36285.75  36312.60  25625   
..                  ...       ...       ...       ...       ...    ...   
371 2022-03-21 15:26:00  36129.20  36132.50  36112.20  36112.20  12675   
372 2022-03-21 15:27:00  36116.95  36120.00  36113.30  36113.30  13850   
373 2022-03-21 15:28:00  36114.95  36124.40  36111.60  36123.00  17200   
374 2022-03-21 15:29:00  36122.95  36129.00  36117.75  36125.05  20225   
375 2022-03-21 15:30:00  36120.05  36120.05  36120.05  36120.05    100   

          oi  
0    2003250  
1    2003250  
2    2037275  
3    2037275  
4    2037275  
..       ...  
371  2

(2022-03-22 06:41:05,199) WARNING :: Disconnected from Historical Data REST Connection ! (PID:4836 Thread:2832)


### Main code

In [8]:
import pickle
from sklearn.preprocessing import MinMaxScaler
df33 = hist_data['c']
df44 = hist_data['v']

df33 = df33.values[-31:-1]
df44 = df44.values[-31:-1]

df33 = df33.reshape((len(df33), 1))
df44 = df44.reshape((len(df44), 1))

df33_scaled = scaler.transform(df33)
df44_scaled = scaler1.transform(df44)

dataset_stacked_test = np.hstack((df33_scaled, df44_scaled))
dataset_stacked_test.shape
print(dataset_stacked_test)


[[1.13235926 0.01693564]
 [1.1327269  0.02885157]
 [1.13141305 0.01379246]
 [1.12997465 0.05409082]
 [1.13058537 0.03199475]
 [1.13097108 0.03246388]
 [1.13129854 0.00867893]
 [1.13147734 0.01247889]
 [1.13075211 0.02378495]
 [1.1311539  0.01102458]
 [1.13146327 0.0239726 ]
 [1.13171238 0.01102458]
 [1.13289565 0.02172077]
 [1.13407289 0.03377744]
 [1.13453897 0.04128354]
 [1.13505125 0.04437981]
 [1.13481821 0.04700694]
 [1.13372133 0.02350347]
 [1.13384588 0.02143929]
 [1.13388003 0.02036029]
 [1.13381776 0.04015763]
 [1.13464946 0.02279977]
 [1.13502111 0.03344905]
 [1.13541286 0.05751548]
 [1.13544902 0.05624883]
 [1.13505125 0.03546632]
 [1.13433606 0.02378495]
 [1.13438026 0.02598987]
 [1.13477    0.03227622]
 [1.13485236 0.03795271]]


In [9]:
X_test_5 = dataset_stacked_test[-6:-1]
X_test_5 = np.expand_dims(X_test_5, axis=0)
X_test_5 = np.asarray(X_test_5)

X_test_10 = dataset_stacked_test[-11:-1]
X_test_10 = np.expand_dims(X_test_10, axis=0)
X_test_10 = np.asarray(X_test_10)

X_test_15 = dataset_stacked_test[-16:-1]
X_test_15 = np.expand_dims(X_test_15, axis=0)
X_test_15 = np.asarray(X_test_15)

X_test_20 = dataset_stacked_test[-21:-1]
X_test_20 = np.expand_dims(X_test_20, axis=0)
X_test_20 = np.asarray(X_test_20)

X_test_25 = dataset_stacked_test[-26:-1]
X_test_25 = np.expand_dims(X_test_25, axis=0)
X_test_25 = np.asarray(X_test_25)

X_test_30 = dataset_stacked_test[-31:-1]
X_test_30 = np.expand_dims(X_test_30, axis=0)
X_test_30 = np.asarray(X_test_30)

In [10]:
pd.options.mode.chained_assignment = None  

In [11]:
IST = pytz.timezone('Asia/Kolkata') 
cols = ['Time','Prediction_5','Prediction_10','Prediction_15','Prediction_20','Prediction_25','Prediction_30','Actual']
df = pd.DataFrame(columns=cols, index=range(80))
model1 = tf.keras.models.load_model("BANKNIFY_11year_5min_qty_ltp.hdf5")
model2 = tf.keras.models.load_model("BANKNIFY_11year_10min_qty_ltp.hdf5")
model3 = tf.keras.models.load_model("BANKNIFY_11year_15min_qty_ltp.hdf5")
model4 = tf.keras.models.load_model("BANKNIFY_11year_20min_qty_ltp.hdf5")
model5 = tf.keras.models.load_model("BANKNIFY_11year_25min_qty_ltp.hdf5")
model6 = tf.keras.models.load_model("BANKNIFY_11year_30min_qty_ltp.hdf5")
a=0
while True:
    now = datetime.now()
    if dt(9,20,0)<=dt(now.hour,now.minute,0)<=dt(15,25,0) and datetime.today().isoweekday()!=6 and datetime.today().isoweekday()!=7:
        sleep(300 - time() % 300)  # run the code every 5 min
        hist_data_1 = td_app.get_historic_data(symbol, duration='2 D', bar_size=barsize)
        hist_data = pd.DataFrame(hist_data_1)
        if len(hist_data)>=5:
            
            now = datetime.now()  

            df33 = hist_data['c']
            df44 = hist_data['v']

            df33 = df33.values[-31:-1]
            df44 = df44.values[-31:-1]

            df33 = df33.reshape((len(df33), 1))
            df44 = df44.reshape((len(df44), 1))

            df33_scaled = scaler.transform(df33)
            df44_scaled = scaler1.transform(df44)

            dataset_stacked_test = np.hstack((df33_scaled, df44_scaled))
            
            X_test_5_1 = dataset_stacked_test[-6:-1]
            X_test_5_1 = np.expand_dims(X_test_5_1, axis=0)
            X_test_5_1 = np.asarray(X_test_5_1)
            
            X_test_10_1 = dataset_stacked_test[-11:-1]
            X_test_10_1 = np.expand_dims(X_test_10_1, axis=0)
            X_test_10_1 = np.asarray(X_test_10_1)
            
            X_test_15_1 = dataset_stacked_test[-16:-1]
            X_test_15_1 = np.expand_dims(X_test_15_1, axis=0)
            X_test_15_1 = np.asarray(X_test_15_1)
            
            X_test_20_1 = dataset_stacked_test[-21:-1]
            X_test_20_1 = np.expand_dims(X_test_20_1, axis=0)
            X_test_20_1 = np.asarray(X_test_20_1)
            
            X_test_25_1 = dataset_stacked_test[-26:-1]
            X_test_25_1 = np.expand_dims(X_test_25_1, axis=0)
            X_test_25_1 = np.asarray(X_test_25_1)
            
            X_test_30_1 = dataset_stacked_test[-31:-1]
            X_test_30_1 = np.expand_dims(X_test_30_1, axis=0)
            X_test_30_1 = np.asarray(X_test_30_1)
            
            y_5 = X_test_5_1[0][4][0]
            y_5 = y_5.reshape((1, 1))
            Y_test_5 = np.expand_dims(y_5, axis=0) 
            Y_test_5 =  np.asarray(Y_test_5)

            model1.fit(X_test_5,Y_test_5,epochs=50,batch_size=1,verbose=0)
            model1.save("BANKNIFY_11year_5min_qty_ltp.hdf5")
            
            Y_test_5_predict = model1.predict(X_test_5_1)
            Y_test_5_predict = scaler.inverse_transform(Y_test_5_predict)
            datetime_ist = datetime.now(IST)
            print("Date & Time in IST : ", 
            datetime_ist.strftime('%Y:%m:%d %H:%M:%S'))
            print("5 min Prediction: " + str(Y_test_5_predict))
            X_test_5 = X_test_5_1
            df.Time.loc[a] =  datetime_ist.strftime('%Y:%m:%d %H:%M:%S')
            df.Prediction_5.loc[a] = Y_test_5_predict
            df.Actual.loc[a]= hist_data['c'].values[-1]
                        
            if (now.minute%10) == 0:
                if len(hist_data)>=10:
                    y_10 = X_test_10_1[0][9][0]
                    y_10 = y_10.reshape((1, 1))
                    Y_test_10 = np.expand_dims(y_10, axis=0) 
                    Y_test_10 =  np.asarray(Y_test_10)

                    model2.fit(X_test_10,Y_test_10,epochs=50,batch_size=1,verbose=0)
                    model2.save("BANKNIFY_11year_10min_qty_ltp.hdf5")

                    Y_test_10_predict = model2.predict(X_test_10_1)
                    Y_test_10_predict = scaler.inverse_transform(Y_test_10_predict)
                    datetime_ist = datetime.now(IST)
                    print("Date & Time in IST : ", 
                    datetime_ist.strftime('%Y:%m:%d %H:%M:%S'))
                    print("10 min Prediction: " + str(Y_test_10_predict))
                    X_test_10 = X_test_10_1
                    df.Prediction_10.loc[a] = Y_test_10_predict
                
            if (now.minute%15) == 0:
                if len(hist_data)>=15:
                    y_15 = X_test_15_1[0][14][0]
                    y_15 = y_15.reshape((1, 1))
                    Y_test_15 = np.expand_dims(y_15, axis=0) 
                    Y_test_15 =  np.asarray(Y_test_15)

                    model3.fit(X_test_15,Y_test_15,epochs=50,batch_size=1,verbose=0)
                    model3.save("BANKNIFY_11year_15min_qty_ltp.hdf5")

                    Y_test_15_predict = model3.predict(X_test_15_1)
                    Y_test_15_predict = scaler.inverse_transform(Y_test_15_predict)
                    datetime_ist = datetime.now(IST)
                    print("Date & Time in IST : ", 
                    datetime_ist.strftime('%Y:%m:%d %H:%M:%S'))
                    print("15 min Prediction: " + str(Y_test_15_predict))
                    X_test_15 = X_test_15_1
                    df.Prediction_15.loc[a] = Y_test_15_predict
                
            if (now.minute%20) == 0:
                if len(hist_data)>=20:
                    y_20 = X_test_20_1[0][19][0]
                    y_20 = y_20.reshape((1, 1))
                    Y_test_20 = np.expand_dims(y_20, axis=0) 
                    Y_test_20 =  np.asarray(Y_test_20)

                    model4.fit(X_test_20,Y_test_20,epochs=50,batch_size=1,verbose=0)
                    model4.save("BANKNIFY_11year_20min_qty_ltp.hdf5")

                    Y_test_20_predict = model4.predict(X_test_20_1)
                    Y_test_20_predict = scaler.inverse_transform(Y_test_20_predict)
                    datetime_ist = datetime.now(IST)
                    print("Date & Time in IST : ", 
                    datetime_ist.strftime('%Y:%m:%d %H:%M:%S'))
                    print("20 min Prediction: " + str(Y_test_20_predict))
                    X_test_20 = X_test_20_1
                    df.Prediction_20.loc[a] = Y_test_20_predict
                
            if (now.minute%25) == 0:
                if len(hist_data)>=25:
                    y_25 = X_test_25_1[0][23][0]
                    y_25 = y_25.reshape((1, 1))
                    Y_test_25 = np.expand_dims(y_25, axis=0) 
                    Y_test_25 =  np.asarray(Y_test_25)

                    model5.fit(X_test_25,Y_test_25,epochs=50,batch_size=1,verbose=0)
                    model5.save("BANKNIFY_11year_25min_qty_ltp.hdf5")

                    Y_test_25_predict = model5.predict(X_test_25_1)
                    Y_test_25_predict = scaler.inverse_transform(Y_test_25_predict)
                    datetime_ist = datetime.now(IST)
                    print("Date & Time in IST : ", 
                    datetime_ist.strftime('%Y:%m:%d %H:%M:%S'))
                    print("25 min Prediction: " + str(Y_test_25_predict))
                    X_test_25 = X_test_25_1
                    df.Prediction_25.loc[a] = Y_test_25_predict
                
            if (now.minute%30) == 0:
                if len(hist_data)>=30:
                    y_30 = X_test_30_1[0][14][0]
                    y_30 = y_30.reshape((1, 1))
                    Y_test_30 = np.expand_dims(y_30, axis=0) 
                    Y_test_30 =  np.asarray(Y_test_30)

                    model6.fit(X_test_30,Y_test_30,epochs=50,batch_size=1,verbose=0)
                    model6.save("BANKNIFY_11year_30min_qty_ltp.hdf5")

                    Y_test_30_predict = model6.predict(X_test_30_1)
                    Y_test_30_predict = scaler.inverse_transform(Y_test_30_predict)
                    datetime_ist = datetime.now(IST)
                    print("Date & Time in IST : ", 
                    datetime_ist.strftime('%Y:%m:%d %H:%M:%S'))
                    print("30 min Prediction: " + str(Y_test_30_predict))
                    X_test_30 = X_test_30_1  
                    df.Prediction_30.loc[a] = Y_test_30_predict
                
            a=a+1
            df.to_csv('BANKNIFTY_'+str(now.day)+'-'+str(now.month)+'-'+str(now.year)+'.csv')

    else:
        cols = ['Time','Actual','Prediction_5','Prediction_10','Prediction_15','Prediction_20','Prediction_25','Prediction_30','Actual']
        df = pd.DataFrame(columns=cols, index=range(80)) 
        a=0

Date & Time in IST :  2022:03:22 09:25:05
5 min Prediction: [[35911.945]]
Date & Time in IST :  2022:03:22 09:25:08
25 min Prediction: [[35878.367]]
Date & Time in IST :  2022:03:22 09:30:02
5 min Prediction: [[35812.477]]
Date & Time in IST :  2022:03:22 09:30:05
10 min Prediction: [[35711.598]]
Date & Time in IST :  2022:03:22 09:30:09
15 min Prediction: [[35683.37]]
Date & Time in IST :  2022:03:22 09:30:12
30 min Prediction: [[35919.434]]
Date & Time in IST :  2022:03:22 09:35:02
5 min Prediction: [[35746.414]]
Date & Time in IST :  2022:03:22 09:40:02
5 min Prediction: [[35794.758]]
Date & Time in IST :  2022:03:22 09:40:02
10 min Prediction: [[35766.598]]
Date & Time in IST :  2022:03:22 09:40:05
20 min Prediction: [[35571.7]]
Date & Time in IST :  2022:03:22 09:45:02
5 min Prediction: [[35727.66]]
Date & Time in IST :  2022:03:22 09:45:02
15 min Prediction: [[35700.027]]
Date & Time in IST :  2022:03:22 09:50:02
5 min Prediction: [[35727.57]]
Date & Time in IST :  2022:03:22 09:

(2022-03-23 09:25:01,111) WARNING :: Connected successfully to TrueData Historical Data Service...  (PID:4836 Thread:2832)


Date & Time in IST :  2022:03:23 09:25:03
5 min Prediction: [[36908.45]]
Date & Time in IST :  2022:03:23 09:25:04
25 min Prediction: [[36915.88]]
Date & Time in IST :  2022:03:23 09:30:02
5 min Prediction: [[36774.77]]
Date & Time in IST :  2022:03:23 09:30:02
10 min Prediction: [[36849.863]]
Date & Time in IST :  2022:03:23 09:30:03
15 min Prediction: [[36884.066]]
Date & Time in IST :  2022:03:23 09:30:04
30 min Prediction: [[36755.12]]
Date & Time in IST :  2022:03:23 09:35:02
5 min Prediction: [[36714.453]]
Date & Time in IST :  2022:03:23 09:40:02
5 min Prediction: [[36724.344]]
Date & Time in IST :  2022:03:23 09:40:02
10 min Prediction: [[36712.527]]
Date & Time in IST :  2022:03:23 09:40:03
20 min Prediction: [[36890.062]]
Date & Time in IST :  2022:03:23 09:45:02
5 min Prediction: [[36673.15]]
Date & Time in IST :  2022:03:23 09:45:03
15 min Prediction: [[36628.61]]
Date & Time in IST :  2022:03:23 09:50:02
5 min Prediction: [[36629.777]]
Date & Time in IST :  2022:03:23 09:5

(2022-03-24 09:25:01,105) WARNING :: Connected successfully to TrueData Historical Data Service...  (PID:4836 Thread:2832)


Date & Time in IST :  2022:03:24 09:25:03
5 min Prediction: [[35631.895]]
Date & Time in IST :  2022:03:24 09:25:04
25 min Prediction: [[35530.992]]
Date & Time in IST :  2022:03:24 09:30:02
5 min Prediction: [[35674.035]]
Date & Time in IST :  2022:03:24 09:30:03
10 min Prediction: [[35498.355]]
Date & Time in IST :  2022:03:24 09:30:03
15 min Prediction: [[35462.71]]
Date & Time in IST :  2022:03:24 09:30:04
30 min Prediction: [[35836.62]]
Date & Time in IST :  2022:03:24 09:35:02
5 min Prediction: [[35748.953]]
Date & Time in IST :  2022:03:24 09:40:02
5 min Prediction: [[35794.46]]
Date & Time in IST :  2022:03:24 09:40:03
10 min Prediction: [[35802.703]]
Date & Time in IST :  2022:03:24 09:40:03
20 min Prediction: [[35560.207]]
Date & Time in IST :  2022:03:24 09:45:02
5 min Prediction: [[35778.32]]
Date & Time in IST :  2022:03:24 09:45:02
15 min Prediction: [[35784.27]]
Date & Time in IST :  2022:03:24 09:50:02
5 min Prediction: [[35788.3]]
Date & Time in IST :  2022:03:24 09:50

...

Date & Time in IST :  2022:03:25 09:25:03
5 min Prediction: [[35448.38]]
Date & Time in IST :  2022:03:25 09:25:04
25 min Prediction: [[35497.33]]
Date & Time in IST :  2022:03:25 09:30:01
5 min Prediction: [[35465.848]]
Date & Time in IST :  2022:03:25 09:30:02
10 min Prediction: [[35439.027]]
Date & Time in IST :  2022:03:25 09:30:03
15 min Prediction: [[35443.45]]
Date & Time in IST :  2022:03:25 09:30:04
30 min Prediction: [[35537.227]]
Date & Time in IST :  2022:03:25 09:35:01
5 min Prediction: [[35489.406]]
Date & Time in IST :  2022:03:25 09:40:02
5 min Prediction: [[35493.496]]
Date & Time in IST :  2022:03:25 09:40:02
10 min Prediction: [[35498.207]]
Date & Time in IST :  2022:03:25 09:40:03
20 min Prediction: [[35415.555]]
Date & Time in IST :  2022:03:25 09:45:02
5 min Prediction: [[35545.613]]
Date & Time in IST :  2022:03:25 09:45:02
15 min Prediction: [[35544.465]]
Date & Time in IST :  2022:03:25 09:50:02
5 min Prediction: [[35512.8]]
Date & Time in IST :  2022:03:25 09:

(2022-03-28 09:25:01,094) WARNING :: Connected successfully to TrueData Historical Data Service...  (PID:4836 Thread:2832)


Date & Time in IST :  2022:03:28 09:25:02
5 min Prediction: [[35227.145]]
Date & Time in IST :  2022:03:28 09:30:01
5 min Prediction: [[35148.715]]
Date & Time in IST :  2022:03:28 09:30:02
10 min Prediction: [[35090.918]]
Date & Time in IST :  2022:03:28 09:35:01
5 min Prediction: [[35152.05]]
Date & Time in IST :  2022:03:28 09:40:01
5 min Prediction: [[35191.414]]
Date & Time in IST :  2022:03:28 09:40:02
10 min Prediction: [[35178.543]]
Date & Time in IST :  2022:03:28 09:40:02
20 min Prediction: [[35047.42]]
Date & Time in IST :  2022:03:28 09:45:01
5 min Prediction: [[35204.715]]
Date & Time in IST :  2022:03:28 09:45:02
15 min Prediction: [[35052.56]]
Date & Time in IST :  2022:03:28 09:50:01
5 min Prediction: [[35121.805]]
Date & Time in IST :  2022:03:28 09:50:02
10 min Prediction: [[35131.062]]
Date & Time in IST :  2022:03:28 09:50:03
25 min Prediction: [[34909.465]]
Date & Time in IST :  2022:03:28 09:55:01
5 min Prediction: [[35208.97]]
Date & Time in IST :  2022:03:28 10:

(2022-03-28 11:05:54,231) ERROR :: <class 'requests.exceptions.ConnectionError'> -> ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None)) (PID:4836 Thread:2832)


UnboundLocalError: local variable 'hist_data' referenced before assignment

In [15]:
import requests
endpoint = "http://nimblerest.lisuns.com:4531/"
apikey = "8d2239ae-1856-4e2d-b447-77cc3d278711"

In [16]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler1 = MinMaxScaler(feature_range=(0,1))

In [22]:
async def GetLastQuote():
    print("----------------------------------------------------")
    print("Work in progress... sending request for GetLastQuote")
    print("----------------------------------------------------")
    ExchangeName = "NFO"
    InstIdentifier = "BANKNIFTY-I"
    isShortIdentifier = "false"         #GFDL : When using contractwise symbol like NIFTY20JULFUT, 
    Periodicity = "Minute"                                   #this argument must be sent with value "true" 
    xml="false"                                        
    response = ""
    c=0
    count = 1                  #Change this number to see more updates. By default, sample will print 10 updates and stop
    maxi=5
    global x
    while(c<count):
        strMessage = endpoint+"GetHistory/?accessKey="+apikey+"&exchange="+ExchangeName+"&instrumentIdentifier="+InstIdentifier+"&periodicity="+ Periodicity+"&max="+str(maxi)+"&xml="+xml
        try:
            response = requests.get(strMessage,verify=False)
            print("Message sent : "+strMessage)
            #print("Waiting for response...")
            x = response.text
            #print("Response :\n" + x)
            #print("----------------------------------------------------")
            print("Data Fatched Successfully")
        except:
            time.sleep(2)
            try:
                response = requests.get(strMessage,verify=False)
                #print("Message sent : "+strMessage)
                #print("Waiting for response...")
                x = response.text
                #print("Response :\n" + x)
                #print("----------------------------------------------------")
                print("Data Fatched Successfully")
            except:
                pass
                
        c = c+1
await GetLastQuote()


----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Message sent : http://nimblerest.lisuns.com:4531/GetHistory/?accessKey=8d2239ae-1856-4e2d-b447-77cc3d278711&exchange=NFO&instrumentIdentifier=BANKNIFTY-I&periodicity=Minute&max=5&xml=false
Data Fatched Successfully


In [23]:
async def GetLastQuote375():
    print("----------------------------------------------------")
    print("Work in progress... sending request for GetLastQuote")
    print("----------------------------------------------------")
    ExchangeName = "NFO"
    InstIdentifier = "BANKNIFTY-I"
    isShortIdentifier = "false"         #GFDL : When using contractwise symbol like NIFTY20JULFUT, 
    Periodicity = "Minute"                                   #this argument must be sent with value "true" 
    xml="false"                                        
    response = ""
    c=0
    count = 1                  #Change this number to see more updates. By default, sample will print 10 updates and stop
    maxi=30
    global y
    while(c<count):
        strMessage = endpoint+"GetHistory/?accessKey="+apikey+"&exchange="+ExchangeName+"&instrumentIdentifier="+InstIdentifier+"&periodicity="+ Periodicity+"&max="+str(maxi)+"&xml="+xml
        try:
            response = requests.get(strMessage,verify=False)
            #print("Message sent : "+strMessage)
            #print("Waiting for response...")
            y = response.text
            #print("Response :\n" + y)
            print("Data Fatched Successfully")
            #print("----------------------------------------------------")
        except:
            time.sleep(2)
            try:
                response = requests.get(strMessage,verify=False)
                #print("Message sent : "+strMessage)
                #print("Waiting for response...")
                y = response.text
                #print("Response :\n" + y)
                print("Data Fatched Successfully")
                #print("----------------------------------------------------")
            except:
                pass
                
        c = c+1
await GetLastQuote375()

----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully


In [24]:
async def GetLastTick50():
    print("----------------------------------------------------")
    print("Work in progress... sending request for GetLastQuote")
    print("----------------------------------------------------")
    ExchangeName = "NFO"
    InstIdentifier = "BANKNIFTY-I"
    isShortIdentifier = "false"         #GFDL : When using contractwise symbol like NIFTY20JULFUT, 
    Periodicity = "Tick"                                   #this argument must be sent with value "true" 
    xml="false"                                        
    response = ""
    c=0
    count = 1                  #Change this number to see more updates. By default, sample will print 10 updates and stop
    maxi=250
    global z
    while(c<count):
        strMessage = endpoint+"GetHistory/?accessKey="+apikey+"&exchange="+ExchangeName+"&instrumentIdentifier="+InstIdentifier+"&periodicity="+ Periodicity+"&max="+str(maxi)+"&xml="+xml
        try:
            response = requests.get(strMessage,verify=False)
            #print("Message sent : "+strMessage)
            #print("Waiting for response...")
            z = response.text
            #print("Response :\n" + z)
            print("Data Fatched Successfully")
            #print("----------------------------------------------------")
        except:
            time.sleep(2)
            try:
                response = requests.get(strMessage,verify=False)
                #print("Message sent : "+strMessage)
                #print("Waiting for response...")
                z = response.text
                #print("Response :\n" + z)
                print("Data Fatched Successfully")
                #print("----------------------------------------------------")
            except:
                pass
                
        c = c+1
await GetLastTick50()

----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully


In [25]:
# ExchangeName = "NFO"
# InstIdentifier = "BANKNIFTY-I"
# isShortIdentifier = "false"         #GFDL : When using contractwise symbol like NIFTY20JULFUT, 
# Periodicity = "Tick"                                   #this argument must be sent with value "true" 
# xml="false"                                        
# response = ""
# fr = 1642563900
# to = 1642586400
# strMessage = endpoint+"GetHistory/?accessKey="+apikey+"&exchange="+ExchangeName+"&instrumentIdentifier="+InstIdentifier+"&periodicity="+ Periodicity+"&from="+str(fr)+"&to="+str(to)+"&xml="+xml
# response = requests.get(strMessage,verify=False)
# #print("Message sent : "+strMessage)
# #print("Waiting for response...")
# z = response.text
# #print("Response :\n" + z)
# print("Data Fatched Successfully")
# #print("----------------------------------------------------")
# data = json.loads(z)
# df_main = pd.json_normalize(data['TICK'])
# df_main = df_main.iloc[::-1]
# df_main
# #df_main.to_csv('Tick_6thDec_19thDec.csv')

In [26]:
import json
import pandas as pd
data = json.loads(x)
df_main = pd.json_normalize(data['OHLC'])
df_main = df_main.iloc[::-1]

df3 = df_main['CLOSE'] 
df4 = df_main['TRADEDQTY']

df33 = df3.values[0:]
df44 = df4.values[0:]/25

print(df33)
print(df44)

Qty_past = df4.values[-1]

df33 = df33.reshape((len(df33), 1))
df44 = df44.reshape((len(df44), 1))

df33_scaled = scaler1.fit_transform(df33)
df44_scaled = scaler.fit_transform(df44)

dataset_stacked_test = np.hstack((df33_scaled, df44_scaled))

X_test = dataset_stacked_test
X_test = np.expand_dims(X_test, axis=0)
X_test_np =  np.asarray(X_test)

cols = ['Time','Prediction','Volume','Logic','Signal']
df = pd.DataFrame(columns=cols, index=range(400))
r=10

[38140.95 38181.75 38156.9  38174.05 38165.7 ]
[2390. 1254.  806.  854.  613.]


In [27]:
IST = pytz.timezone('Asia/Kolkata') 

model1 = tf.keras.models.load_model("Stock_model_BANNIFTY_5min_1min_qty_1.hdf5")
a = 0
    
while True:
    now = datetime.now()  
    if dt(9,15,0)<=dt(now.hour,now.minute,0)<=dt(15,30,0) and datetime.today().isoweekday()!=6 and datetime.today().isoweekday()!=7:
        
        sleep(60 - time() % 60)  
        await GetLastQuote()
        await GetLastTick50()
        
        try:  
            data1 = json.loads(x)
            df_main1 = pd.json_normalize(data1['OHLC'])
            df_main1 = df_main1.iloc[::-1]
            print(df_main1)

            df3 = df_main1['CLOSE']
            df4 = df_main1['TRADEDQTY']

            df33 = df3.values[0:]
            df44 = df4.values[0:]/25

            Qty_present = df4.values[-1]/25

            df33 = df33.reshape((5, 1))
            df44 = df44.reshape((5, 1))

            df33_scaled = scaler1.fit_transform(df33)
            df44_scaled = scaler.fit_transform(df44)

            dataset_stacked_test_1 = np.hstack((df33_scaled, df44_scaled))
            dataset_stacked_test_1 = np.expand_dims(dataset_stacked_test_1, axis=0)
            dataset_stacked_test_1 =  np.asarray(dataset_stacked_test_1)
            
            df33_1 =  df33_scaled[-1]
            df33_1 = df33_1.reshape((1, 1))

            Y_test = np.expand_dims(df33_1, axis=0) 
            Y_test_np =  np.asarray(Y_test)

            model1.fit(X_test_np,Y_test_np,epochs=50,batch_size=1,verbose=0)

            Y_test_np_predict = model1.predict(dataset_stacked_test_1)
            Y_test_np_predict = scaler1.inverse_transform(Y_test_np_predict)
            datetime_ist = datetime.now(IST)
            print("Date & Time in IST : ", datetime_ist.strftime('%Y:%m:%d %H:%M'))
            
            last_50 = json.loads(z)
            df_50 = pd.json_normalize(last_50['TICK'])
            df_50 = df_50.iloc[::-1]
            
            v=1
            for k in range(2):
                if ((df_main1.TRADEDQTY[0]-df_main1.TRADEDQTY[k+1])/25) > 265:
                    v=0
            df.Volume.loc[a] = df_main1.TRADEDQTY[0]    ##
            print("Volume Difference: "+str((df_main1.TRADEDQTY[0]-df_main1.TRADEDQTY[1])/25))
            df.Logic.loc[a] = str(df_main1.TRADEDQTY[0]/25)+"-"+str(df_main1.TRADEDQTY[1]/25)+":"+str((df_main1.TRADEDQTY[0]-df_main1.TRADEDQTY[1])/25) ##
            if v==0:
                diff=[]
                for l in range(len(df_50)-1):
                    diff.append(df_50.LASTTRADEPRICE[0]-df_50.LASTTRADEPRICE[l+1])
                S1 = sum(s>0 for s in diff)
                S2 = sum(s<0 for s in diff)
                if S1>0.95*len(df_50):
                    df.Signal.loc[a] = "Sell"    ##
                    print(str(Y_test_np_predict)+" Sell")
                elif S2<0.95*len(df_50):
                    df.Signal.loc[a] = "Buy"    ##
                    print(str(Y_test_np_predict)+" Buy")
                else:
                    print("Next Predicted Value"+str(Y_test_np_predict))
                    print("Turning Point")
                    df.Signal.loc[a] = "Turning Point"
            else:
                print("Next Predicted Value"+str(Y_test_np_predict))  
                
                    
            df.Prediction.loc[a] = Y_test_np_predict[0][0]
            df.Time.loc[a] =  datetime_ist.strftime('%Y:%m:%d %H:%M')
            
            a = a+1
            
            df.to_csv('BANKNIFTY_'+str(now.day)+'-'+str(now.month)+'-'+str(now.year)+'.csv')
            X_test_np = dataset_stacked_test_1
            Qty_past = Qty_present
            model1.save("Stock_model_BANNIFTY_5min_1min_qty_1.hdf5")
        except:
            await GetLastQuote()
            await GetLastTick50()
            
            try:
                data1 = json.loads(x)
                df_main1 = pd.json_normalize(data1['OHLC'])
                df_main1 = df_main1.iloc[::-1]
                print(df_main1)

                df3 = df_main1['CLOSE']
                df4 = df_main1['TRADEDQTY']

                df33 = df3.values[0:]
                df44 = df4.values[0:]/25

                Qty_present = df4.values[-1]/25

                df33 = df33.reshape((5, 1))
                df44 = df44.reshape((5, 1))

                df33_scaled = scaler1.fit_transform(df33)
                df44_scaled = scaler.fit_transform(df44)

                dataset_stacked_test_1 = np.hstack((df33_scaled, df44_scaled))
                dataset_stacked_test_1 = np.expand_dims(dataset_stacked_test_1, axis=0)
                dataset_stacked_test_1 =  np.asarray(dataset_stacked_test_1)

                df33_1 =  df33_scaled[-1]
                df33_1 = df33_1.reshape((1, 1))

                Y_test = np.expand_dims(df33_1, axis=0) 
                Y_test_np =  np.asarray(Y_test)

                model1.fit(X_test_np,Y_test_np,epochs=50,batch_size=1,verbose=0)

                Y_test_np_predict = model1.predict(dataset_stacked_test_1)
                Y_test_np_predict = scaler1.inverse_transform(Y_test_np_predict)
                datetime_ist = datetime.now(IST)
                print("Date & Time in IST : ", datetime_ist.strftime('%Y:%m:%d %H:%M'))

                last_50= json.loads(z)
                df_50 = pd.json_normalize(last_50['TICK'])
                df_50 = df_50.iloc[::-1]
                
                v=1
                for k in range(2):
                    if ((df_main1.TRADEDQTY[0]-df_main1.TRADEDQTY[k+1])/25) > 265:
                        v=0
                df.Volume.loc[a] = df_main1.TRADEDQTY[0]     #############
                print("Volume Difference: "+str((df_main1.TRADEDQTY[0]-df_main1.TRADEDQTY[1])/25))
                df.Logic.loc[a] = str(df_main1.TRADEDQTY[0]/25)+"-"+str(df_main1.TRADEDQTY[1]/25)+":"+str((df_main1.TRADEDQTY[0]-df_main1.TRADEDQTY[1])/25) ##
                if v==0:
                    diff=[]
                    for l in range(len(df_50)-1):
                        diff.append(df_50.LASTTRADEPRICE[0]-df_50.LASTTRADEPRICE[l+1])
                    S1 = sum(s>0 for s in diff)   
                    S2 = sum(s<0 for s in diff)
                    if S1>0.95*len(df_50):
                        df.Signal.loc[a] = "Sell"    ##
                        print(str(Y_test_np_predict)+" Sell")
                    elif S2<0.95*len(df_50):
                        df.Signal.loc[a] = "Buy"    ##
                        print(str(Y_test_np_predict)+" Buy")
                    else:
                        print("Next Predicted Value"+str(Y_test_np_predict))
                        print("Turning point")
                        df.Signal.loc[a] = "Turning Point"
                    
                else:
                     print("Next Predicted Value"+str(Y_test_np_predict))     
                df.Prediction.loc[a] = Y_test_np_predict[0][0]
                df.Time.loc[a] =  datetime_ist.strftime('%Y:%m:%d %H:%M')
                
                a=a+1
                
                df.to_csv('BANKNIFTY_'+str(now.day)+'-'+str(now.month)+'-'+str(now.year)+'.csv')
                X_test_np = dataset_stacked_test_1
                Qty_past = Qty_present
                model1.save("Stock_model_BANNIFTY_5min_1min_qty_1.hdf5")
            except:
                continue
            
    else:
        cols = ['Time','Prediction','Volume','Logic','Signal']
        df = pd.DataFrame(columns=cols, index=range(400)) 
        #model1.save("Stock_model_BANNIFTY_5min_1min_qty_1.hdf5")
        a=0

----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Message sent : http://nimblerest.lisuns.com:4531/GetHistory/?accessKey=8d2239ae-1856-4e2d-b447-77cc3d278711&exchange=NFO&instrumentIdentifier=BANKNIFTY-I&periodicity=Minute&max=5&xml=false
Data Fatched Successfully
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully
      CLOSE      HIGH  LASTTRADETIME       LOW      OPEN  OPENINTEREST  \
4  38181.75  38210.00  1642650360000  38153.15  38159.75       1975425   
3  38156.90  38190.00  1642650420000  38156.90  38185.00       2007475   
2  38174.05  38178.50  1642650480000  38140.25  38146.30       2007475   
1  38177.55  38180.00  1642650540000  38151.70  38179.00       2007475   
0  38182.45  38182.45  1642650600000  38182.45  38182.45       2007475 

0            25        100  
Date & Time in IST :  2022:01:20 09:32
Volume Difference: -811.0
Next Predicted Value[[38177.21]]
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Message sent : http://nimblerest.lisuns.com:4531/GetHistory/?accessKey=8d2239ae-1856-4e2d-b447-77cc3d278711&exchange=NFO&instrumentIdentifier=BANKNIFTY-I&periodicity=Minute&max=5&xml=false
Data Fatched Successfully
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully
      CLOSE      HIGH  LASTTRADETIME       LOW      OPEN  OPENINTEREST  \
4  38145.00  38158.65  1642651140000  38141.75  38150.25       2021250   
3  38180.00  38182.05  1642651200000  38145.40  38145.40       2021250   
2  38177.00  38200.60  1642651260000  38164.20  38182.95       2021250   
1  38191.30  38200

0            25        125  
Date & Time in IST :  2022:01:20 09:44
Volume Difference: -148.0
Next Predicted Value[[38214.277]]
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Message sent : http://nimblerest.lisuns.com:4531/GetHistory/?accessKey=8d2239ae-1856-4e2d-b447-77cc3d278711&exchange=NFO&instrumentIdentifier=BANKNIFTY-I&periodicity=Minute&max=5&xml=false
Data Fatched Successfully
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully
      CLOSE     HIGH  LASTTRADETIME      LOW      OPEN  OPENINTEREST  \
4  38215.35  38242.5  1642651860000  38211.2  38242.50       2044700   
3  38219.00  38225.0  1642651920000  38202.4  38215.95       2044700   
2  38216.45  38232.0  1642651980000  38214.0  38216.30       2044700   
1  38211.15  38220.0  164

0            25       9775  
Date & Time in IST :  2022:01:20 09:57
Volume Difference: -727.0
Next Predicted Value[[38129.164]]
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Message sent : http://nimblerest.lisuns.com:4531/GetHistory/?accessKey=8d2239ae-1856-4e2d-b447-77cc3d278711&exchange=NFO&instrumentIdentifier=BANKNIFTY-I&periodicity=Minute&max=5&xml=false
Data Fatched Successfully
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully
      CLOSE      HIGH  LASTTRADETIME       LOW      OPEN  OPENINTEREST  \
4  38110.00  38116.10  1642652580000  38099.95  38101.20       2026600   
3  38093.95  38117.00  1642652640000  38079.20  38106.50       2026600   
2  38103.85  38108.60  1642652700000  38073.10  38085.80       2026600   
1  38136.55  3813

0            25        200  
Date & Time in IST :  2022:01:20 10:10
Volume Difference: -305.0
Next Predicted Value[[38090.58]]
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Message sent : http://nimblerest.lisuns.com:4531/GetHistory/?accessKey=8d2239ae-1856-4e2d-b447-77cc3d278711&exchange=NFO&instrumentIdentifier=BANKNIFTY-I&periodicity=Minute&max=5&xml=false
Data Fatched Successfully
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully
      CLOSE      HIGH  LASTTRADETIME       LOW      OPEN  OPENINTEREST  \
4  38096.85  38096.85  1642653420000  38081.80  38082.00       2021900   
3  38098.00  38111.80  1642653480000  38086.60  38090.55       2022925   
2  38095.40  38097.95  1642653540000  38077.65  38082.15       2022925   
1  38063.70  38112

...

0            25        175  
Date & Time in IST :  2022:01:25 13:34
Volume Difference: -427.0
Next Predicted Value[[37385.816]]
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Message sent : http://nimblerest.lisuns.com:4531/GetHistory/?accessKey=8d2239ae-1856-4e2d-b447-77cc3d278711&exchange=NFO&instrumentIdentifier=BANKNIFTY-I&periodicity=Minute&max=5&xml=false
Data Fatched Successfully
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully
      CLOSE      HIGH  LASTTRADETIME       LOW      OPEN  OPENINTEREST  \
4  37414.00  37425.55  1643097660000  37410.00  37421.00       1518175   
3  37390.00  37426.00  1643097720000  37381.00  37414.00       1518175   
2  37380.65  37415.00  1643097780000  37374.00  37390.00       1521150   
1  37387.95  3739

Data Fatched Successfully
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully
     CLOSE      HIGH  LASTTRADETIME       LOW      OPEN  OPENINTEREST  \
4  37420.0  37420.00  1643098380000  37398.65  37412.20       1516075   
3  37393.4  37418.75  1643098440000  37392.00  37417.15       1516075   
2  37369.6  37404.55  1643098500000  37360.00  37399.00       1516625   
1  37363.9  37375.85  1643098560000  37350.60  37368.25       1516625   
0  37365.6  37365.60  1643098620000  37365.60  37365.60       1516625   

   QUOTATIONLOT  TRADEDQTY  
4            25       4050  
3            25       3250  
2            25       5575  
1            25       4500  
0            25        125  
Date & Time in IST :  2022:01:25 13:47
Volume Difference: -175.0
Next Predicted Value[[37385.55]]
----------------------------------------------------
Work in progress... sendi

0            25        225  
Date & Time in IST :  2022:01:25 13:53
Volume Difference: -199.0
Next Predicted Value[[37491.652]]
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Message sent : http://nimblerest.lisuns.com:4531/GetHistory/?accessKey=8d2239ae-1856-4e2d-b447-77cc3d278711&exchange=NFO&instrumentIdentifier=BANKNIFTY-I&periodicity=Minute&max=5&xml=false
Data Fatched Successfully
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully
      CLOSE      HIGH  LASTTRADETIME       LOW      OPEN  OPENINTEREST  \
4  37510.00  37516.00  1643098800000  37481.20  37481.20       1514775   
3  37487.30  37511.95  1643098860000  37481.20  37504.35       1507075   
2  37489.60  37504.00  1643098920000  37482.80  37486.30       1507075   
1  37483.00  3749

0            25        325  
Date & Time in IST :  2022:01:25 14:05
Volume Difference: -875.0
Next Predicted Value[[37506.234]]
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Message sent : http://nimblerest.lisuns.com:4531/GetHistory/?accessKey=8d2239ae-1856-4e2d-b447-77cc3d278711&exchange=NFO&instrumentIdentifier=BANKNIFTY-I&periodicity=Minute&max=5&xml=false
Data Fatched Successfully
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully
      CLOSE      HIGH  LASTTRADETIME       LOW      OPEN  OPENINTEREST  \
4  37518.65  37519.75  1643099520000  37458.85  37458.85       1507975   
3  37535.05  37541.55  1643099580000  37520.00  37529.70       1502350   
2  37540.00  37565.00  1643099640000  37534.85  37535.45       1502350   
1  37535.00  3754

AttributeError: 'builtin_function_or_method' object has no attribute 'sleep'

In [ ]:
IST = pytz.timezone('Asia/Kolkata') 

model1 = tf.keras.models.load_model("Stock_model_BANNIFTY_5min_1min_qty_1.hdf5")
a = 0
    
while True:
    now = datetime.now()  
    if dt(9,15,0)<=dt(now.hour,now.minute,0)<=dt(15,30,0) and datetime.today().isoweekday()!=6 and datetime.today().isoweekday()!=7:
        
        sleep(60 - time() % 60)  
        await GetLastQuote()
        await GetLastTick50()

        data1 = json.loads(x)
        df_main1 = pd.json_normalize(data1['OHLC'])
        df_main1 = df_main1.iloc[::-1]
        print(df_main1)

        df3 = df_main1['CLOSE']
        df4 = df_main1['TRADEDQTY']

        df33 = df3.values[0:]
        df44 = df4.values[0:]/25

        Qty_present = df4.values[-1]/25

        df33 = df33.reshape((5, 1))
        df44 = df44.reshape((5, 1))

        df33_scaled = scaler1.fit_transform(df33)
        df44_scaled = scaler.fit_transform(df44)

        dataset_stacked_test_1 = np.hstack((df33_scaled, df44_scaled))
        dataset_stacked_test_1 = np.expand_dims(dataset_stacked_test_1, axis=0)
        dataset_stacked_test_1 =  np.asarray(dataset_stacked_test_1)

        df33_1 =  df33_scaled[-1]
        df33_1 = df33_1.reshape((1, 1))

        Y_test = np.expand_dims(df33_1, axis=0) 
        Y_test_np =  np.asarray(Y_test)

        model1.fit(X_test_np,Y_test_np,epochs=50,batch_size=1,verbose=0)

        Y_test_np_predict = model1.predict(dataset_stacked_test_1)
        Y_test_np_predict = scaler1.inverse_transform(Y_test_np_predict)
        datetime_ist = datetime.now(IST)
        print("Date & Time in IST : ", datetime_ist.strftime('%Y:%m:%d %H:%M'))

        last_50 = json.loads(z)
        df_50 = pd.json_normalize(last_50['TICK'])
        df_50 = df_50.iloc[::-1]

        v=1
        for k in range(2):
            if ((df_main1.TRADEDQTY[0]-df_main1.TRADEDQTY[k+1])/25) > 265:
                v=0
        df.Volume.loc[a] = df_main1.TRADEDQTY[0]    ##
        print("Volume Difference: "+str((df_main1.TRADEDQTY[0]-df_main1.TRADEDQTY[1])/25))
        df.Logic.loc[a] = str(df_main1.TRADEDQTY[0]/25)+"-"+str(df_main1.TRADEDQTY[1]/25)+":"+str((df_main1.TRADEDQTY[0]-df_main1.TRADEDQTY[1])/25) ##
        if v==0:
            diff=[]
            for l in range(len(df_50)-1):
                diff.append(df_50.LASTTRADEPRICE[0]-df_50.LASTTRADEPRICE[l+1])
                S1 = sum(s>0 for s in diff)
                print('s1'+str(S1))
                S2 = sum(s<0 for s in diff)
                print('s2'+str(S2))
            if S1>0.95*len(df_50):
                df.Signal.loc[a] = "Buy"    ##
                print(str(Y_test_np_predict)+" Buy")
            elif S2<0.95*len(df_50):
                df.Signal.loc[a] = "Sell"    ##
                print(str(Y_test_np_predict)+" Sell")
            else:
                print("Next Predicted Value"+str(Y_test_np_predict))
                print("Turning Point")
        else:
            print("Next Predicted Value"+str(Y_test_np_predict))  


        df.Prediction.loc[a] = Y_test_np_predict[0][0]
        df.Time.loc[a] =  datetime_ist.strftime('%Y:%m:%d %H:%M')

        a = a+1

        df.to_csv('BANKNIFTY_'+str(now.day)+'-'+str(now.month)+'-'+str(now.year)+'.csv')
        X_test_np = dataset_stacked_test_1
        Qty_past = Qty_present
        model1.save("Stock_model_BANNIFTY_5min_1min_qty_1.hdf5")

    else:
        cols = ['Time','Prediction','Volume','Logic','Signal']
        df = pd.DataFrame(columns=cols, index=range(400)) 
        a=0


----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Message sent : http://nimblerest.lisuns.com:4531/GetHistory/?accessKey=8d2239ae-1856-4e2d-b447-77cc3d278711&exchange=NFO&instrumentIdentifier=BANKNIFTY-I&periodicity=Minute&max=5&xml=false
Data Fatched Successfully
----------------------------------------------------
Work in progress... sending request for GetLastQuote
----------------------------------------------------
Data Fatched Successfully
      CLOSE      HIGH  LASTTRADETIME       LOW     OPEN  OPENINTEREST  \
4  36872.00  36872.00  1641286320000  36864.25  36870.0       2270575   
3  36867.60  36872.00  1641286380000  36865.00  36872.0       2270575   
2  36878.00  36878.00  1641286440000  36864.80  36865.0       2272025   
1  36867.05  36890.65  1641286500000  36865.45  36880.0       2272025   
0  36848.40  36872.30  1641286560000  36835.85  36872.3       2272025   

  

In [39]:
!pip install beautifulsoup4

In [42]:
# Python program to modify HTML
# with the help of Beautiful Soup

# Import the libraries
from bs4 import BeautifulSoup as bs
import os
import re


# Open the HTML in which you want to make changes
html = open('index.html')

# Parse HTML file in Beautiful Soup
soup = bs(html, 'html.parser')

# Give location where text is
# stored which you wish to alter
old_text = soup.find("p", {"id": "para"})

# Replace the already stored text with
# the new text which you wish to assign
# new_text = old_text.find(text=re.compile(
# 	'Geeks For Geeks')).replace_with('Vinayak Rai')

for i in soup.find("p", {"id": "para"}).findChildren():
    i.replace_with('##')

# Alter HTML file to see the changes done
with open("index.html", "wb") as f_output:
	f_output.write(soup.prettify("utf-8"))
